In [ ]:
!nvidia-smi

In [ ]:
!pip list

In [ ]:
!pip install ultralytics roboflow

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks


## Load Dataset

Get the code from roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY_GOES_HERE")
project = rf.workspace("test-mhm3s").project("kudzu-in-gsv")
version = project.version(3)
dataset = version.download("yolov8")
                

## Train Model

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=20 imgsz=512

#### Confusion Matrix

In [ ]:
Image(filename='/Users/alba/KudzuClassification/ObjectDetection/runs/detect/train/confusion_matrix.png', width=600)

#### Train Results

In [ ]:
Image(filename='/Users/alba/KudzuClassification/ObjectDetection/runs/detect/train/results.png', width=600)

## Validation

In [ ]:
!yolo task=detect mode=val model=/Users/alba/KudzuClassification/ObjectDetection/weights/best_v3.pt data={dataset.location}/data.yaml

## Predict Images

In [ ]:
import os 
import shutil
from math import ceil
import json

source_folder = '/Users/alba/KudzuClassification/ObjectDetection/output'

image_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

In [ ]:
import os
import subprocess

# Define the path where your subfolders are located
base_directory = "/Users/alba/KudzuClassification/ObjectDetection/output"  # Change this to your base folder containing 68 subfolders
model_path = "/Users/alba/KudzuClassification/ObjectDetection/weights/best_v3.pt"
conf_threshold = 0.95  # Set confidence threshold to 95%
processed_log = "processed_images_v3.txt"  # Log file to track processed images
detections_log = "detections_above_threshold_v3.txt"  # Log file to track images with detections above threshold


# Function to load already processed images from log file
def load_processed_images(log_file):
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            return set(line.strip() for line in f)
    return set()

def count_processed_images(processed_set):
    return len(processed_set)
    
# Function to append new processed image to log file
def log_processed_image(log_file, image_path):
    with open(log_file, 'a') as f:
        f.write(f"{image_path}\n")


In [ ]:
# Load the list of already processed images
processed_images = load_processed_images(processed_log)
processed_count = count_processed_images(processed_images)
# Print the processed count after all images are processed
print(f"Skipping {processed_count} images already processed.")

# Process each image in the subdirectory
for image_file in image_files:
    image_path = os.path.join(base_directory, image_file)

    # Check if this image has already been processed (skip if true)
    if image_path in processed_images:    
        continue
        
    

    # Construct the YOLO command with confidence threshold
    yolo_command = [
        "yolo", 
        "task=detect", 
        "mode=predict", 
        f"model={model_path}", 
        f"source={image_path}",
        f"conf={conf_threshold}"  # Set confidence threshold to 90%
    ]

    # Run the YOLO command using subprocess and capture output
    #print(f"Running YOLO on {image_path}...")
    result = subprocess.run(yolo_command, capture_output=True, text=True)

    # Get YOLO's raw output for analysis
    output = result.stdout
    #print(f"YOLO Output for {image_path}:\n{output}")

    # Check if there is a Kudzu detection by looking for 'kudzu' in the output
    if "kudzu" in output:  # Adjust this based on actual YOLO output structure
        print(f"Kudzu detected for {image_path} with confidence above {conf_threshold * 100}%")

        # Log this image as having Kudzu detections above the threshold
        with open(detections_log, 'a') as f:
            f.write(f"{image_path}\n")

    # Mark this image as processed by logging it
    log_processed_image(processed_log, image_path)


print("YOLO detection completed for all images.")


### Split Images into Folders

In [ ]:
import os
import shutil
from math import ceil
import json

# Define the source folder and destination folder
source_folder = "/home/student/Desktop/KudzuClassification/ObjectDetection/output"
destination_folder = "/home/student/Desktop/KudzuClassification/ObjectDetection/predict"

# Get a list of all image files in the source folder
image_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]


In [ ]:
def find_divisors(n):
    divisors = set()  # Using a set to avoid duplicates
    for i in range(1, int(n**0.5) + 1):  # Iterate up to the square root of n
        if n % i == 0:  # If i is a divisor
            divisors.add(i)  # Add i
            divisors.add(n // i)  # Add the corresponding divisor n // i
    return sorted(divisors)  # Return the divisors in sorted order

divisors = find_divisors(len(image_files))
print(f"The divisors of {len(image_files)} are: {divisors}")

In [ ]:
# Define the number of subfolders you want to create, chose one of the divisiors
num_subfolders = 68  # Change this to however many subfolders you want

# Calculate how many images should be in each subfolder
images_per_folder = ceil(len(image_files) / num_subfolders)
print(images_per_folder)

In [ ]:
# Create the destination subfolders and distribute the images
for i in range(num_subfolders):
    # Create a new subfolder
    subfolder_path = os.path.join(destination_folder, f"subfolder_{i+1}")
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Get the start and end index for this batch of images
    start_index = i * images_per_folder
    end_index = start_index + images_per_folder
    
    # Move or copy the images to the new subfolder
    for image_file in image_files[start_index:end_index]:
        src_path = os.path.join(source_folder, image_file)
        dst_path = os.path.join(subfolder_path, image_file)
        shutil.copy(src_path, dst_path)  # Use shutil.move(src_path, dst_path) if you want to move instead of copy

print("Images have been successfully distributed into subfolders.")

In [ ]:
import os
import subprocess

# Define the path where your subfolders are located
base_directory = "/home/student/Desktop/KudzuClassification/ObjectDetection/predict/"  # Change this to your base folder containing 68 subfolders
model_path = "/home/student/Desktop/KudzuClassification/ObjectDetection/weights/best_v2.pt"
conf_threshold = 0.95  # Set confidence threshold to 95%
processed_log = "processed_images_v2.txt"  # Log file to track processed images
detections_log = "detections_above_threshold_v2.txt"  # Log file to track images with detections above threshold


In [ ]:
# Function to load already processed images from log file
def load_processed_images(log_file):
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            return set(line.strip() for line in f)
    return set()

# Function to append new processed image to log file
def log_processed_image(log_file, image_path):
    with open(log_file, 'a') as f:
        f.write(f"{image_path}\n")


In [ ]:
# Load the list of already processed images
processed_images = load_processed_images(processed_log)

# Get a list of all subdirectories in the base directory
subdirectories = [f.path for f in os.scandir(base_directory) if f.is_dir()]

processed_count = 0

# Loop over each subdirectory and process its images
for subdir in subdirectories:
    # List all image files in the current subdirectory (assuming common image formats like .jpg, .jpeg, .png, etc.)
    image_files = [f for f in os.listdir(subdir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Process each image in the subdirectory
    for image_file in image_files:
        image_path = os.path.join(subdir, image_file)

        # Check if this image has already been processed (skip if true)
        if image_path in processed_images:
            processed_count += 1    
            continue
            
        print(f"Skipping {processed_count} images already processed.")
        
        # Construct the YOLO command with confidence threshold
        yolo_command = [
            "yolo", 
            "task=detect", 
            "mode=predict", 
            f"model={model_path}", 
            f"source={image_path}",
            f"conf={conf_threshold}"  # Set confidence threshold to 90%
        ]
        
        # Run the YOLO command using subprocess and capture output
        #print(f"Running YOLO on {image_path}...")
        result = subprocess.run(yolo_command, capture_output=True, text=True)
        
        # Get YOLO's raw output for analysis
        output = result.stdout
        #print(f"YOLO Output for {image_path}:\n{output}")
        
        # Check if there is a Kudzu detection by looking for 'kudzu' in the output
        if "kudzu" in output:  # Adjust this based on actual YOLO output structure
            print(f"Kudzu detected for {image_path} with confidence above {conf_threshold * 100}%")
            
            # Log this image as having Kudzu detections above the threshold
            with open(detections_log, 'a') as f:
                f.write(f"{image_path}\n")
        
        # Mark this image as processed by logging it
        log_processed_image(processed_log, image_path)

print("YOLO detection completed for all images.")

In [ ]:
import os
import shutil
from pathlib import Path

# Paths
text_file_path = 'detections_above_threshold_v2.txt'  # Update with actual path
root_directory = '/home/student/Desktop/runs/detect/'  # Update if necessary

# Ensure output directory exists
output_directory = '/home/student/Desktop/matching_images_v2'  # Change this as needed
os.makedirs(output_directory, exist_ok=True)

# Step 1: Read image paths from the text file and extract filenames
try:
    with open(text_file_path, 'r') as file:
        # Extract only the filenames from the full paths in the text file
        listed_image_filenames = {os.path.basename(line.strip()) for line in file.readlines()}
    print(f"Loaded {len(listed_image_filenames)} image filenames from text file.")
except FileNotFoundError:
    print('Text file not found! Check the file path.')
    listed_image_filenames = set()

In [ ]:
# Step 2: Check if root directory exists and process images
try:
    predict_folders = [d for d in Path(root_directory).iterdir() if d.is_dir() and d.name.startswith('predict')]
    print(f"Found {len(predict_folders)} 'predict' folders.")

    for folder in predict_folders:
        print(f"Processing folder: {folder}")
        for file in folder.glob('*.*'):  # Match all files with any extension
            if file.name in listed_image_filenames:  # Compare only filenames
                print(f'Copying: {file}')
                shutil.copy(str(file), output_directory)
except FileNotFoundError:
    print('Root directory not found! Check the directory path.')

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Specify the output directory where your images are saved
output_directory = '/home/student/Desktop/matching_images_v2'

# Step 1: Fetch all image file paths from the output directory
if os.path.exists(output_directory):
    image_files = [os.path.join(output_directory, f) for f in os.listdir(output_directory) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    
    # Step 2: Randomly select 10 images from the list (or fewer if less than 10)
    if len(image_files) > 10:
        random_images = random.sample(image_files, 10)
    else:
        random_images = image_files  # Select all if less than 10
    
    # Step 3: Display the randomly selected images using matplotlib
    plt.figure(figsize=(30, 20))
    
    for i, img_path in enumerate(random_images):
        # Load and display each image
        print(i+1, img_path)
        img = mpimg.imread(img_path)
        plt.subplot(2, 5, i + 1)  # Create a grid of 2 rows and 5 columns
        plt.imshow(img)
        plt.axis('off')  # Hide axis for better visualization
        plt.title(os.path.basename(img_path))  # Display filename as title
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Directory does not exist: {output_directory}")

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Specify the output directory where your images are saved
output_directory = '/home/student/Desktop/matching_images_v2'

# Step 1: Fetch all image file paths from the output directory
if os.path.exists(output_directory):
    image_files = [os.path.join(output_directory, f) for f in os.listdir(output_directory) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    
    # Step 2: Display images in batches of 10
    batch_size = 10
    total_batches = (len(image_files) + batch_size - 1) // batch_size  # Calculate total batches
    
    batch_index = 0
    while batch_index < total_batches:
        # Get the images for the current batch
        start_index = batch_index * batch_size
        end_index = start_index + batch_size
        current_batch = image_files[start_index:end_index]
        
        # Display the current batch
        plt.figure(figsize=(30, 20))
        for i, img_path in enumerate(current_batch):
            img = mpimg.imread(img_path)
            print(i+1, img_path)
            plt.subplot(2, 5, i + 1)  # Create a grid of 2 rows and 5 columns
            plt.imshow(img)
            plt.axis('off')  # Hide axis for better visualization
            plt.title(os.path.basename(img_path))  # Display filename as title
        
        plt.tight_layout()
        plt.show()
        
        # Ask user whether to proceed to the next batch
        if batch_index < total_batches - 1:
            user_input = input("Press Enter to view the next batch, or type 'q' to quit: ").strip().lower()
            if user_input == 'q':
                print("Exiting...")
                break
        
        batch_index += 1
else:
    print(f"Directory does not exist: {output_directory}")
